# Logins per day from Extended Event session

This will get the logins per day from the Extended Event session


In [ ]:
SELECT CAST(GetDate() AS nvarchar(20)) AS 'Execution Time'

In [ ]:
SELECT
    CAST(n.value('(@timestamp)[1]', 'datetime2') AS Date) AS LoginDate,
    n.value('(@timestamp)[1]', 'datetime2') AS [utc_timestamp],
    n.value('(action[@name="session_nt_username"]/value)[1]', 'nvarchar(128)') as nt_username,
    n.value('(action[@name="client_hostname"]/value)[1]', 'nvarchar(128)') as client_hostname
INTO #tempxeresults
from (select cast(event_data as XML) as event_data
from sys.fn_xe_file_target_read_file('MonitorWindowsLogins*.xel', null, null, null)) ed
cross apply ed.event_data.nodes('event') as q(n)

BEGIN TRY
SELECT 
LoginDate
,COUNT(DISTINCT nt_username) AS NumberOfLogins
FROM #tempxeresults 
GROUP BY LoginDate
ORDER BY LoginDate
DROP TABLE #tempxeresults
END TRY
BEGIN CATCH

DROP TABLE #tempxeresults
END CATCH